# Iris con clasificadores generativos

Al igual que los árboles de decisión, es muy fácil aprender y evaluar clasificadores generativos con sklearn.

In [2]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score

Lectura del corpus iris:

In [3]:
iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.4, shuffle=True, random_state=23)

El clasificador naive Bayes Gaussiano es muy fácil de ajustar ya que, básicamente, basta hallar la media y varianza empíricas por cada par clase-dimensión separadamente. No obstante, en la práctica solemos encontrarnos con varianzas empíricas (quasi-)nulas para algunos pares clase-dimensión que ocasionan inestabilidad en los cálculos. Para reducir esta inestabildad se añade una porción de la mayor varianza empírica hallada a todas las varianzas empíricas. Esta porción se especifica mediante el parámetro var_smoothing (1e-9 por omisión).

In [4]:
clf = GaussianNB(var_smoothing=1e-9).fit(X_train, y_train)
acc = accuracy_score(y_test, clf.predict(X_test))
print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de GaussianNB() es 93.3%


**Ejercicio:** Estudia el efecto de var_smoothing sobre la precisión de naive Bayes Gaussiano.

In [6]:
for vs in [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 10]:
    clf = GaussianNB(var_smoothing=vs).fit(X_train, y_train)
    acc = accuracy_score(y_test, clf.predict(X_test))
    print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de GaussianNB() es 93.3%
La precisión de GaussianNB(var_smoothing=1e-08) es 93.3%
La precisión de GaussianNB(var_smoothing=1e-07) es 93.3%
La precisión de GaussianNB(var_smoothing=1e-06) es 93.3%
La precisión de GaussianNB(var_smoothing=1e-05) es 93.3%
La precisión de GaussianNB(var_smoothing=0.0001) es 93.3%
La precisión de GaussianNB(var_smoothing=0.001) es 93.3%
La precisión de GaussianNB(var_smoothing=0.01) es 95.0%
La precisión de GaussianNB(var_smoothing=0.1) es 95.0%
La precisión de GaussianNB(var_smoothing=1) es 91.7%
La precisión de GaussianNB(var_smoothing=10) es 70.0%


LDA ajusta una Gaussiana a cada clase asumiendo que todas las clases comparten la misma matriz de covarianzas. Al igual que la estimación de varianzas empíricas en naive Bayes Gaussiano, la estimación de la matriz de covarianzas en LDA se regulariza apropiadamente. En concreto, la técnica de regularización por omisión emplea un parámetro de tolerancia, tol (1e-4 por omisión), que controla su fuerza.

In [30]:
clf = LinearDiscriminantAnalysis(tol=1e-4).fit(X_train, y_train)
acc = accuracy_score(y_test, clf.predict(X_test))
print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de LinearDiscriminantAnalysis() es 98.3%


**Ejercicio:** Estudia el efecto de tol sobre la precisión de LDA.

In [9]:
for t in [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1]:
    clf = LinearDiscriminantAnalysis(tol=t).fit(X_train, y_train)
    acc = accuracy_score(y_test, clf.predict(X_test))
    print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de LinearDiscriminantAnalysis(tol=1e-09) es 98.3%
La precisión de LinearDiscriminantAnalysis(tol=1e-08) es 98.3%
La precisión de LinearDiscriminantAnalysis(tol=1e-07) es 98.3%
La precisión de LinearDiscriminantAnalysis(tol=1e-06) es 98.3%
La precisión de LinearDiscriminantAnalysis(tol=1e-05) es 98.3%
La precisión de LinearDiscriminantAnalysis() es 98.3%
La precisión de LinearDiscriminantAnalysis(tol=0.001) es 98.3%
La precisión de LinearDiscriminantAnalysis(tol=0.01) es 98.3%
La precisión de LinearDiscriminantAnalysis(tol=0.1) es 98.3%
La precisión de LinearDiscriminantAnalysis(tol=1) es 30.0%


QDA ajusta una Gaussiana independiente a cada clase. Emplea el mismo parámetro de tolerancia que LDA para regularizar la estimación de matrices de covarianzas, aunque en este caso no afecta a las predicciones, sino solo a los avisos que se producen cuando se considera que las muestras son colineales:

In [10]:
for tol in [1e-4, .5, .9]:
    clf = QuadraticDiscriminantAnalysis(tol=tol).fit(X_train, y_train)
    acc = accuracy_score(y_test, clf.predict(X_test))
    print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de QuadraticDiscriminantAnalysis() es 98.3%
La precisión de QuadraticDiscriminantAnalysis(tol=0.5) es 98.3%
La precisión de QuadraticDiscriminantAnalysis(tol=0.9) es 98.3%


C:\Users\pablo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\pablo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


La regularización en QDA se lleva a cabo mediante el parámetro reg_param (0.0 por omisión). QDA interpola la matriz de covarianzas empírica con la identidad en función de reg_param, de 0.0 (matriz empírica) a 1.0 (identidad).

**Ejercicio:** Estudia el efecto de reg_param sobre la precisión de QDA.

In [16]:
for rp in [1e-9, 1e-6, 1e-3, 0.1,1]:
    for tol in [1e-4, .5, .9]:
        clf = QuadraticDiscriminantAnalysis(tol=tol,reg_param=rp).fit(X_train, y_train)
        acc = accuracy_score(y_test, clf.predict(X_test))
        print('La precisión de {0!s} es {1:.1%}'.format(clf, acc))

La precisión de QuadraticDiscriminantAnalysis(reg_param=1e-09) es 98.3%
La precisión de QuadraticDiscriminantAnalysis(reg_param=1e-09, tol=0.5) es 98.3%
La precisión de QuadraticDiscriminantAnalysis(reg_param=1e-09, tol=0.9) es 98.3%
La precisión de QuadraticDiscriminantAnalysis(reg_param=1e-06) es 98.3%
La precisión de QuadraticDiscriminantAnalysis(reg_param=1e-06, tol=0.5) es 98.3%
La precisión de QuadraticDiscriminantAnalysis(reg_param=1e-06, tol=0.9) es 98.3%
La precisión de QuadraticDiscriminantAnalysis(reg_param=0.001) es 98.3%
La precisión de QuadraticDiscriminantAnalysis(reg_param=0.001, tol=0.5) es 98.3%
La precisión de QuadraticDiscriminantAnalysis(reg_param=0.001, tol=0.9) es 98.3%
La precisión de QuadraticDiscriminantAnalysis(reg_param=0.1) es 95.0%
La precisión de QuadraticDiscriminantAnalysis(reg_param=0.1, tol=0.5) es 95.0%
La precisión de QuadraticDiscriminantAnalysis(reg_param=0.1, tol=0.9) es 95.0%
La precisión de QuadraticDiscriminantAnalysis(reg_param=1) es 93.3%
La

C:\Users\pablo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\pablo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\pablo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\pablo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn